<a href="https://colab.research.google.com/github/hurgj123kr/project2/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 먼저 제가 2000~2020년간 나온 노래들을 가지고 메타크리틱 AOTY사이트 두 곳에서 평점을 낸 데이터들입니다. 이 데이터들을 가지고 메타크리틱의 유저점수와 AOTY사이트의 유저점수 두 가지 모델을 회귀모델로 만들어서 성능이 좋은 모델을 만들어보고 평소에 평점을 내는 사이트에 관심이 있었는데 음악이 그렇게 생소한 분야도 아니기도해서 이 데이터셋을 사용하게되었습니다.

## 아직

### 먼저 데이터를 불러와서 데이터의 타입, 결측치, 중복치, 카디널리티 확인한다.

In [ ]:
import pandas as pd
url = '/Users/sunghoheo/Downloads/music.csv'
df = pd.read_csv(url)
df.head(5)

,Artist,Title,Release Month,Release Day,Release Year,Format,Label,Genre,Metacritic Critic Score,Metacritic Reviews,Metacritic User Score,Metacritic User Reviews,AOTY Critic Score,AOTY Critic Reviews,AOTY User Score,AOTY User Reviews
0,Neko Case,Middle Cyclone,March,3,2009,LP,ANTI-,Alt-Country,79.0,31.0,8.7,31.0,79,25,78,55
1,Jason Isbell & The 400 Unit,Jason Isbell & The 400 Unit,February,17,2009,LP,Thirty Tigers,Country Rock,70.0,14.0,8.4,7.0,73,11,73,8
2,Animal Collective,Merriweather Post Pavilion,January,20,2009,LP,Domino,Psychedelic Pop,89.0,36.0,8.5,619.0,92,30,87,1335
3,Bruce Springsteen,Working on a Dream,January,27,2009,LP,Columbia Records,Rock,72.0,29.0,7.9,101.0,70,23,66,38
4,Andrew Bird,Noble Beast,January,20,2009,LP,Fat Possum,Singer-Songwriter,79.0,29.0,8.7,47.0,74,24,78,44


In [ ]:
df.dtypes

Artist                      object
Title                       object
Release Month               object
Release Day                 object
Release Year                 int64
Format                      object
Label                       object
Genre                       object
Metacritic Critic Score    float64
Metacritic Reviews         float64
Metacritic User Score      float64
Metacritic User Reviews    float64
AOTY Critic Score            int64
AOTY Critic Reviews          int64
AOTY User Score              int64
AOTY User Reviews            int64
dtype: object

In [ ]:
df.isnull().sum()

Artist                         0
Title                          1
Release Month                589
Release Day                  590
Release Year                   0
Format                         0
Label                       3417
Genre                       7960
Metacritic Critic Score    18474
Metacritic Reviews         18474
Metacritic User Score      23259
Metacritic User Reviews    21784
AOTY Critic Score              0
AOTY Critic Reviews            0
AOTY User Score                0
AOTY User Reviews              0
dtype: int64

In [ ]:
df.T.duplicated()

Artist                     False
Title                      False
Release Month              False
Release Day                False
Release Year               False
Format                     False
Label                      False
Genre                      False
Metacritic Critic Score    False
Metacritic Reviews         False
Metacritic User Score      False
Metacritic User Reviews    False
AOTY Critic Score          False
AOTY Critic Reviews        False
AOTY User Score            False
AOTY User Reviews          False
dtype: bool

In [ ]:
df.describe(exclude='number')#숫자형데이터 확인
df.describe(exclude='object')#오브젝트형데이터 즉 문자열데이터임
df.describe(exclude='float').T.sort_values(by='unique')

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Format,32358,2,LP,30470,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Release Month,31769,12,October,3553,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Release Day,31768,39,1,2458,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genre,24398,293,Hip Hop,1714,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Label,28941,8250,Columbia,510,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artist,32358,12914,Miles Davis,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Title,32357,30231,II,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Release Year,32358.0,NaN,NaN,NaN,2006.66781,14.132255,1940.0,2002.0,2012.0,2017.0,2020.0
AOTY Critic Score,32358.0,NaN,NaN,NaN,72.808332,11.214989,4.0,67.0,74.0,80.0,100.0
AOTY Critic Reviews,32358.0,NaN,NaN,NaN,5.645003,6.179734,1.0,2.0,3.0,7.0,48.0


In [ ]:
#features object컬럼의 cardinality확인하는법
#object 타입의 featureas를 고르기
object_data =[]

for col in df.columns:
    if df[col].dtype=='object':
        object_data.append(col)
object_data
int_data = []
float_data =[]
#int리스트 형태로
for col in df.columns:
    if df[col].dtype=='int':
        int_data.append(col)
int_data
#float리스트 형태로 추출
float_data = []
for col in df.columns:
    if df[col].dtype=='float':
        float_data.append(col)

print("실수형:",float_data)
print("object:",object_data)
print("정수형:",int_data)

실수형: ['Metacritic Critic Score', 'Metacritic Reviews', 'Metacritic User Score', 'Metacritic User Reviews']
object: ['Artist', 'Title', 'Release Month', 'Release Day', 'Format', 'Label', 'Genre']
정수형: ['Release Year', 'AOTY Critic Score', 'AOTY Critic Reviews', 'AOTY User Score', 'AOTY User Reviews']


In [ ]:
df[object_data].describe().T.sort_values(by=['unique'])
#전체로
df.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
Format,32358,2,LP,30470
Release Month,31769,12,October,3553
Release Day,31768,39,1,2458
Genre,24398,293,Hip Hop,1714
Label,28941,8250,Columbia,510
Artist,32358,12914,Miles Davis,44
Title,32357,30231,II,23


### 결측치,중복치,카디널리티 확인을 해보니 결측치는 object데이터는 나 월,일,은 unknown으로, 숫자형데이터는 평균값을 넣어서 데체합니다.
### 중복치는 없습니다. 이상치도 확인해보겠습니다.

In [ ]:
#float값들인열 이상치확인
for col in df.iloc[:,8:]:
    print(year_2000[col].describe())
    print('-'*50)

count    13866.000000
mean        73.276792
std          8.397163
min         15.000000
25%         69.000000
50%         74.000000
75%         79.000000
max         98.000000
Name: Metacritic Critic Score, dtype: float64
--------------------------------------------------
count    13866.000000
mean        12.480456
std          7.914760
min          4.000000
25%          6.000000
50%         10.000000
75%         17.000000
max         51.000000
Name: Metacritic Reviews, dtype: float64
--------------------------------------------------
count    9083.000000
mean        7.780634
std         0.978252
min         1.400000
25%         7.400000
50%         8.000000
75%         8.500000
max         9.500000
Name: Metacritic User Score, dtype: float64
--------------------------------------------------
count    10557.000000
mean        63.165293
std        325.458674
min          3.000000
25%          8.000000
50%         14.000000
75%         37.000000
max      14720.000000
Name: Metacritic Use

### 이상치 문제는 없습니다. 이제 결측치 데체 하겠습니다. 값들중에 메타크리틱 유저 점수는 한자릿수단위값이므로 다른 수치들과 똑같이 10단위로 맞춰주겠습니다.

In [ ]:
df['Metacritic User Score'] = df['Metacritic User Score'] * 10
cols = ['Title','Release Month','Release Day','Label','Genre']
for col in cols:
    df[col].fillna('Unknown', inplace=True)

df.isnull().sum()

Artist                         0
Title                          0
Release Month                  0
Release Day                    0
Release Year                   0
Format                         0
Label                          0
Genre                          0
Metacritic Critic Score    18474
Metacritic Reviews         18474
Metacritic User Score      23259
Metacritic User Reviews    21784
AOTY Critic Score              0
AOTY Critic Reviews            0
AOTY User Score                0
AOTY User Reviews              0
dtype: int64

### 이제 수치형값들을 가지고있는 float타입의 열들의 결측치를 최대한 채워주겠습니다.

In [ ]:
#나머지 메타크리틱 비평가 점수~ 메타크리틱 유저 리뷰까지 결측치 처리
#isna()는 결측치를 True Fals로 반환 하는것.
cols = ['Metacritic Critic Score','Metacritic Reviews','Metacritic User Score','Metacritic User Reviews']
for genre in df['Genre'].unique():
        for col in cols2:
            value = round(df.loc[df['Genre']== genre, col].mean(), 0)
            df.loc[(df['Genre']== genre) & (df[col].isna()),col]= value


df.isnull().sum()

Artist                       0
Title                        0
Release Month                0
Release Day                  0
Release Year                 0
Format                       0
Label                        0
Genre                        0
Metacritic Critic Score    197
Metacritic Reviews         197
Metacritic User Score      296
Metacritic User Reviews    263
AOTY Critic Score            0
AOTY Critic Reviews          0
AOTY User Score              0
AOTY User Reviews            0
dtype: int64

In [ ]:
#아직 덜 채워져서 나머지 마저 채우겠습니다.
missing = df[df['Metacritic User Score'].isnull()]
missing.Genre.value_counts()

Folk Metal           23
Roots Reggae         19
French Pop           16
Screamo              15
Industrial Techno    13
Bossa Nova           12
Acid House           12
Minimalism           11
No Wave              10
Neofolk               9
J-Pop                 9
Jazz-Rock             8
Juke                  7
Boom Bap              6
Heavy Psych           6
Protopunk             6
Symphonic Metal       6
Acid Jazz             6
Digital Hardcore      5
AOR                   5
Musique concrÃ¨te     5
Outsider House        5
Acid Techno           4
Latin Alternative     4
Skate Punk            4
Ambient House         4
Yacht Rock            4
Crossover Thrash      4
Psychobilly           4
Jazz-Funk             4
Minimal Wave          4
Dark Jazz             3
YÃ©-yÃ©               3
TropicÃ¡lia           3
Plunderphonics        3
Classical             3
Tropical House        2
Trap Metal            2
Khyal                 2
City Pop              2
UK Garage             2
Dungeon Synth   

In [ ]:
df[df['Genre'] == 'Folk Metal']

,Artist,Title,Release Month,Release Day,Release Year,Format,Label,Genre,Metacritic Critic Score,Metacritic Reviews,Metacritic User Score,Metacritic User Reviews,AOTY Critic Score,AOTY Critic Reviews,AOTY User Score,AOTY User Reviews
3495,Alestorm,Black Sails at Midnight,May,27,2009,LP,Napalm,Folk Metal,NaN,NaN,NaN,NaN,70,1,74,10
8560,Agalloch,Ashes Against the Grain,August,8,2006,LP,The End,Folk Metal,NaN,NaN,NaN,NaN,93,4,84,112
8561,Agalloch,The Mantle,August,13,2002,LP,The End,Folk Metal,NaN,NaN,NaN,NaN,90,2,86,158
8562,Agalloch,Pale Folklore,July,6,1999,LP,The End Records,Folk Metal,NaN,NaN,NaN,NaN,90,2,81,44
8768,Eluveitie,Origins,August,1,2014,LP,Unknown,Folk Metal,NaN,NaN,NaN,NaN,80,3,74,8
11244,Primordial,To the Nameless Dead,November,16,2007,LP,Metal Blade,Folk Metal,NaN,NaN,NaN,NaN,83,3,81,18
15083,Ensiferum,Unsung Heroes,August,24,2012,LP,Spinefarm,Folk Metal,NaN,NaN,NaN,NaN,63,3,63,6
15084,Ensiferum,Victory Songs,April,27,2007,LP,Spinefarm,Folk Metal,NaN,NaN,NaN,NaN,80,2,73,5
15085,Ensiferum,Iron,April,15,2004,LP,Spinefarm,Folk Metal,NaN,NaN,NaN,NaN,80,2,81,6
15086,Ensiferum,Ensiferum,January,7,2001,LP,Spinefarm,Folk Metal,NaN,NaN,NaN,NaN,80,2,85,5


### 확인해보니 인기가없는 Folk Metal이 아직 결측치가 남아있어서 AOTY데이터의 평균값으로 채우겠습니다.

In [ ]:
cols2 = [['Metacritic Critic Score','AOTY Critic Score'],
        ['Metacritic Reviews','AOTY Critic Reviews'],
        ['Metacritic User Score','AOTY User Score'],
        ['Metacritic User Reviews','AOTY User Reviews']]
for genre in df['Genre'].unique():
    for col, x in cols2:
        value = round(df.loc[df['Genre']== genre, x].mean(),0)
        df.loc[(df['Genre']== genre) & (df[col].isna()),col]=value

df.isnull().sum()

Artist                     0
Title                      0
Release Month              0
Release Day                0
Release Year               0
Format                     0
Label                      0
Genre                      0
Metacritic Critic Score    0
Metacritic Reviews         0
Metacritic User Score      0
Metacritic User Reviews    0
AOTY Critic Score          0
AOTY Critic Reviews        0
AOTY User Score            0
AOTY User Reviews          0
dtype: int64

### 이제 feature enginnering 해주겠습니다

In [ ]:
def enginner(en):